In [1]:
from http.server import BaseHTTPRequestHandler,HTTPServer
import cgi
from basededatos import Base , Restaurant ,MenuItem
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
engine=create_engine('sqlite:///restaurantmenu.db')
Base.metadata.bind=engine
DBsession=sessionmaker(bind=engine)
session=DBsession()

In [ ]:
class webserverHandler(BaseHTTPRequestHandler):
    form_html = \
        '''
        <form method='POST' enctype='multipart/form-data' action='/hello'>
        <h2>Que te gusta que te digan guapo?</h2>
        <input name="message" type="text"><input type="submit" value="Submit" >
        </form>
         '''
    def do_GET(self):
        try:
            if self.path.endswith("/edit"):
                idpath=self.path.split("/")[2]
                print(idpath)
                mirest=session.query(Restaurant).filter_by(id=idpath).one()
                if mirest :
                    self.send_response(200)
                    self.send_header("Content-type","text/html")
                    self.end_headers()
                    output="<html><body>"
                    output+="<h1>"
                    output+=mirest.name
                    output+="</h1>"
                    output+="<form method='POST' enctype='multipart/form-data' action=' /restaurant/%s/edit'>"% idpath
                    output+="<inputname='nuevo_nombre' type='text' placeholder='%s'>" % mirest.name
                    output+="<input type='submit' value='renombre'>"
                    output+="</form>"
                    output+="</html></body>"
                    self.wfile.write(bytes(output, 'utf-8'))

            if self.path.endswith("/restaurant"):
                self.send_response(200)
                self.send_header("Content-type","text/html")
                self.end_headers()
                output='<a href="/restaurant/new">Crear un nuevo restaurante</a>'
                output+='</br>'
                output += "<html><body>"
                restaurants = session.query(Restaurant).all()
                for restaurante in restaurants:
                    output+=restaurante.name
                    output+='</br>'
                    output+='<a href="/restaurant/%s/edit">Editar</a>'
                    output+='</br>'
                    output+='<a href="#">Eliminar</a>'
                    output+='</br>'
                output += "<html><body>"
                self.wfile.write(bytes(output, 'utf-8'))
                print(output)
                
                return
            if self.path.endswith("/restaurant/new"):
                self.send_response(200)
                self.send_header('Content-type', 'text/html')
                self.end_headers()
                output = ""
                output += "<html><body>"
                output += "<h1>Crea un restaurante mardito</h1>"
                output += "<form method = 'POST' enctype='multipart/form-data' action = '/restaurant/new'>"
                output += "<input name = 'restaunuevo' type = 'text' placeholder = 'nuevo restaurante nombre' > "
                output += "<input type='submit' value='Create'>"
                output += "</form></body></html>"
                self.wfile.write(bytes(output,'utf-8'))
                return
                
                return
            if self.path.endswith("/hello"):
                self.send_response(200)
                self.send_header("Content-type","text/html")
                self.end_headers()
                output=""
                output += "<html>" \
                          " <body>" \
                          "     Hello!<br>" + self.form_html + \
                          " </body>" \
                          "</html>"
                self.wfile.write(bytes(output, 'utf-8'))
                print(output)
                return
            if self.path.endswith("/ciao"):
                self.send_response(200)
                self.send_header("Content-type","text/html")
                self.end_headers()
                output = ""
                output += "<html>" \
                          " <body>" \
                          "     Ciaoooo<br>" + self.form_html + \
                          " </body>" \
                          "</html>"
                self.wfile.write(bytes(output, 'utf-8'))
                print(output)
                return
        except IOError:
                self.send_error(404,"No se encontro el archivo %s" % self.path)
    def do_POST(self):
        try:
            if self.path.endswith("/edit"):
                ctype, pdict = cgi.parse_header(
               self.headers['content-type'])

                pdict['boundary'] = bytes(pdict['boundary'], "utf-8")

                if ctype == 'multipart/form-data':
                    fields = cgi.parse_multipart(self.rfile, pdict)
                    message= fields.get('nuevo_nombre')
                    nuevonom = Restaurant(name=message[0].decode())
                    idpath=self.path.split("/")[2]
                    mirest=session.query(Restaurant).filter_by(id=idpath).one()
                    if mirest !=[]:
                        mirest.name=nuevonom
                        session.add(mirest)
                        session.commit()
                        self.send_response(301)
                        self.send_header('Content-type', 'text/html')
                        self.send_header('Location', '/restaurant')
                        self.end_headers()
                
            if self.path.endswith("/restaurant/new"):
            # HEADERS are now in dict/json style container
                ctype, pdict = cgi.parse_header(
                   self.headers['content-type'])

            #boundary data needs to be encoded in a binary format
                pdict['boundary'] = bytes(pdict['boundary'], "utf-8")

                if ctype == 'multipart/form-data':
                    fields = cgi.parse_multipart(self.rfile, pdict)
                    message= fields.get('restaunuevo')
                    restaunuevo = Restaurant(name=message[0].decode())
                    session.add(restaunuevo)
                    session.commit()
                    self.send_response(301)
                    self.send_header('Content-type', 'text/html')
                    self.send_header('Location', '/restaurant')
                    self.end_headers()
        except:
            pass
            #output = ""
            #output += "<html><body>"
            #output += " <h2> Que te parece esto ? ;) </h2>"
            # decode it back into a string rather than byte string(b'stuff')
            #output += "<h1> {} </h1>".format(messagecontent[0].decode())
            #output += self.form_html
            #output += "</body></html>"
            #self.wfile.write(output.encode())
            #print(output)
def main():
    try:
        port=8080
        server=HTTPServer(("",port),webserverHandler)
        print("El servidor web esta ejecutandose en el puerto "+str(port))
        server.serve_forever()
    except KeyboardInterrupt:
        print("El servidor ha sido interrumpido por el usuario , cerrando ...")
        server.socket.close()
        
if __name__ == '__main__':
    main()

El servidor web esta ejecutandose en el puerto 8080
%s
%s


----------------------------------------
Exception happened during processing of request from ('127.0.0.1', 50781)
Traceback (most recent call last):
  File "/Users/rosmerodriguez/anaconda3/lib/python3.6/socketserver.py", line 317, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/Users/rosmerodriguez/anaconda3/lib/python3.6/socketserver.py", line 348, in process_request
    self.finish_request(request, client_address)
  File "/Users/rosmerodriguez/anaconda3/lib/python3.6/socketserver.py", line 361, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/Users/rosmerodriguez/anaconda3/lib/python3.6/socketserver.py", line 696, in __init__
    self.handle()
  File "/Users/rosmerodriguez/anaconda3/lib/python3.6/http/server.py", line 418, in handle
    self.handle_one_request()
  File "/Users/rosmerodriguez/anaconda3/lib/python3.6/http/server.py", line 406, in handle_one_request
    method()
  File "<ipython-input-2-7f23660

In [5]:
idpath=2
mirest=session.query(Restaurant).filter_by(id=idpath).one()
mirest